The most efficient engine from databricks

In [0]:
CREATE OR REPLACE TABLE orders (
  order_id     BIGINT,
  customer_id  BIGINT,
  amount       DECIMAL(10,2),
  order_ts     TIMESTAMP,
  status       STRING
) TBLPROPERTIES (
  delta.enableRowTracking   = true,
  delta.enableChangeDataFeed = true
);

-- Seed a few rows
INSERT INTO orders VALUES
  (1, 101, 20.00, TIMESTAMP'2025-10-01 12:00:00', 'paid'),
  (2, 101, 15.50, TIMESTAMP'2025-10-02 13:00:00', 'paid'),
  (3, 102, 99.00, TIMESTAMP'2025-10-03 09:30:00', 'paid'),
  (4, 103, 17.25, TIMESTAMP'2025-10-03 10:15:00', 'refunded'),
  (5, 104, 35.80, TIMESTAMP'2025-10-04 11:45:00', 'paid');

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW mv 
CLUSTER BY AUTO
TRIGGER ON UPDATE AT MOST EVERY INTERVAL 5 MINUTE
AS
SELECT
  customer_id,
  DATE(order_ts) AS order_date,
  SUM(amount) AS total_amount,
  COUNT(*) AS order_cnt
FROM
  orders
GROUP BY
  customer_id,
  DATE(order_ts);

  -- Late correction to an existing row (breaks pure append-only)
UPDATE orders SET amount = amount + 5.00 WHERE order_id = 2;

-- Append-only rows
INSERT INTO orders VALUES
  (6, 101, 12.00, TIMESTAMP'2025-10-05 08:05:00', 'paid');

-- Refresh the MV
REFRESH MATERIALIZED VIEW mv;

In [0]:
-- Check the planning technique that was used
SELECT timestamp,
       message
FROM event_log(TABLE(mv))
WHERE event_type = 'planning_information'
ORDER BY timestamp DESC LIMIT 2;

In [0]:
DESCRIBE EXTENDED mv;

schedule
  { SCHEDULE [ REFRESH ] schedule_clause |
    TRIGGER ON UPDATE [ AT MOST EVERY trigger_interval ] }

schedule_clause
  { EVERY number { HOUR | HOURS | DAY | DAYS | WEEK | WEEKS } |
    CRON cron_string [ AT TIME ZONE timezone_id ] }